In [1]:
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
model_dir = './src/tl_detector/light_classification/traffic_light_classifier/model_v12/'

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))

In [2]:
X_test_path = glob.glob('./src/tl_detector/light_classification/traffic_light_classifier/test_img/*')
X_test_list=[]
RESIZE=32
for im_path in X_test_path:
    im = np.asarray(Image.open(im_path).resize((RESIZE,RESIZE)))
    X_test_list.append(im)

X_test= np.array(X_test_list)
print(X_test.shape)

(13, 32, 32, 3)


In [ ]:
with tf.Session() as session:
    saver = tf.train.import_meta_graph('model_v10.ckpt.meta')
    saver.restore(session, tf.train.latest_checkpoint('.'))

In [ ]:
lights = {0:'Red', 1:'Yellow', 2:'Green'}
x = tf.get_default_graph().get_tensor_by_name('x:0')
drop_out = tf.get_default_graph().get_tensor_by_name('drop_out:0')
predict_op = tf.get_collection('predict_op') 
with tf.Session() as session:
    saver.restore(session, tf.train.latest_checkpoint('.'))
    y_preds = session.run(predict_op, feed_dict={x:X_test, drop_out:1})[0]
    i =0
    for y in y_preds:
        print(lights[y])
        im = Image.open(X_test_path[i])
        plt.imshow(im)
        plt.show()
        i+=1
        print('-----')

In [ ]:

checkpoint = tf.train.get_checkpoint_state(model_dir)
input_checkpoint = checkpoint.model_checkpoint_path
output_node_names = 'x,y,drop_out,logits'
# We precise the file fullname of our freezed graph
absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
output_graph = absolute_model_dir + "/frozen_model.pb"

# We clear devices to allow TensorFlow to control on which device it will load operations
clear_devices = True

# We start a session using a temporary fresh Graph
with tf.Session(graph=tf.Graph()) as sess:
    # We import the meta graph in the current default Graph
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    # We restore the weights
    saver.restore(sess, input_checkpoint)

    # We use a built-in TF helper to export variables to constants
    output_graph_def = tf.graph_util.convert_variables_to_constants(
        sess, # The session is used to retrieve the weights
        tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
        output_node_names.split(",") # The output node names are used to select the usefull nodes
    ) 

    # Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile(output_graph, "wb") as f:
        f.write(output_graph_def.SerializeToString())
    print("%d ops in the final graph." % len(output_graph_def.node))


In [3]:
def load_graph( graph_file, use_xla=True):
    jit_level = 0
    config = tf.ConfigProto()
    if use_xla:
        jit_level = tf.OptimizerOptions.ON_1
        config.graph_options.optimizer_options.global_jit_level = jit_level

    with tf.Session(graph=tf.Graph(), config=config) as sess:
        gd = tf.GraphDef()
        with tf.gfile.Open(graph_file, 'rb') as f:
            data = f.read()
            gd.ParseFromString(data)
        tf.import_graph_def(gd, name='')
        ops = sess.graph.get_operations()
        n_ops = len(ops)
    return sess, ops

In [4]:
sess_tl, base_ops_tl = load_graph("{}/frozen_model.pb".format(model_dir))

In [ ]:
print(sess_tl)
print( base_ops_tl)

In [5]:
graph=sess_tl.graph
lights = {0:'Red', 1:'Yellow', 2:'Green'}

with tf.Session() as session:
    x = graph.get_tensor_by_name('x:0')
    y = graph.get_tensor_by_name('y:0')
    drop_out = graph.get_tensor_by_name('drop_out:0')
    logits = graph.get_tensor_by_name('logits:0')
    predict_op = tf.argmax(logits,1)
#     predict_op = tf.get_collection('y:0') 
    r = sess_tl.run(predict_op, feed_dict={x: X_test, drop_out:1})
    print(r)
    i =0
    for y in r:
        print(lights[y], X_test_path[i])
        i+=1

[1 2 0 0 1 0 2 2 2 2 1 0 2]
('Yellow', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im7.jpg')
('Green', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im6.jpg')
('Red', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im5.jpg')
('Red', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im9.jpeg')
('Yellow', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im8.jpeg')
('Red', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im1.jpg')
('Green', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im2.jpg')
('Green', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im3.jpg')
('Green', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im13.jpg')
('Green', './src/tl_detector/light_classification/traffic_light_classifier/test_img/im11.jpg')
('Yellow', './src/tl_detector/li

In [ ]:
from tensorflow.python.framework.graph_util import convert_variables_to_constants
with tf.gfile.FastGFile("./model_v10/" + "graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()

frozen_graph_def = convert_variables_to_constants(sess, graph_def, ["output"])

with tf.gfile.GFile("./model_v10/" + "frozen.pb", "wb") as f:
    f.write(frozen_graph_def.SerializeToString())

In [ ]:




import sys
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [ ]:
MODEL_NAME = 'model_v11'
root_path = './src/tl_detector/light_classification/traffic_light_classifier/model_v11/'
input_graph_path =  root_path+MODEL_NAME+'.pbtxt'
checkpoint_path = root_path+ 'model_v11.ckpt.'
input_saver_def_path = ""
input_binary = False
output_node_names = "y, x, drop_out"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = root_path+'/frozen_'+MODEL_NAME+'.pb'
output_optimized_graph_name = root_path+'/optimized_'+MODEL_NAME+'.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")



# Optimize for inference

# input_graph_def = tf.GraphDef()
# with tf.gfile.Open(output_frozen_graph_name, "r") as f:
#     data = f.read()
#     input_graph_def.ParseFromString(data)

# output_graph_def = optimize_for_inference_lib.optimize_for_inference(
#         input_graph_def,
#         ["I"], # an array of the input node(s)
#         ["O"], # an array of output nodes
#         tf.float32.as_datatype_enum)

# # Save the optimized graph

# f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
# f.write(output_graph_def.SerializeToString())